# Práctica BiciMad Grupo 20

Por Lucía del Nido Herranz, Víctor Güejes Cepeda y Celia Vaquero Espinosa

En esta práctica vamos a realizar un estudio sobre los datos de BiciMad de todos los meses del año 2018. Para ello vamos a utilizar los DataFrames de Spark en python.


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.functions import col,udf,isnull,collect_list, struct
from pyspark.sql.types import IntegerType, StringType, BooleanType

import warnings
warnings.filterwarnings('ignore')

22/05/22 12:50:39 WARN Utils: Your hostname, ldelnido-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/05/22 12:50:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/22 12:50:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

FILES = ["201801_Usage_Bicimad.json",
         "201802_Usage_Bicimad.json",
         "201803_Usage_Bicimad.json",
         "201804_Usage_Bicimad.json",
         "201805_Usage_Bicimad.json",
         "201806_Usage_Bicimad.json",
         "201807_Usage_Bicimad.json",
         "201808_Usage_Bicimad.json",
         "201809_Usage_Bicimad.json",
         "201810_Usage_Bicimad.json",
         "201811_Usage_Bicimad.json",
         "201812_Usage_Bicimad.json"]


In [53]:
def union_archivos():
    '''
    recibe la lista de archivos FILES y los une en un unico rdd
    '''
    rddlst = []
    for f in FILES:
        df = spark.read.json(f)
        df = df.drop('_corrupt_record','track').dropna()
        rddlst.append(df)
    df = reduce(DataFrame.unionAll, rddlst)
    return df

In [54]:
df = union_archivos()
df = df.drop('_id','unplug_hourTime','idplug_base','idunplug_base','track')
df.show()

+--------+--------------+----------------+-----------+--------------------+---------+--------+
|ageRange|idplug_station|idunplug_station|travel_time|       user_day_code|user_type|zip_code|
+--------+--------------+----------------+-----------+--------------------+---------+--------+
|       5|             7|               6|        284|b00665a845be18ed9...|        1|   28010|
|       0|           117|              24|        666|65a82ac6353d9e8b2...|        1|        |
|       0|           117|              24|        662|51b7da733980586e1...|        1|        |
|       3|           110|              82|        708|0af09bb0e23045fb7...|        1|   28005|
|       4|            58|             169|        171|500e6780660a1fb23...|        1|   28004|
|       5|            74|              92|        427|40816bb3978f9931f...|        1|   28006|
|       0|            18|              34|        166|ed325bd1941a25394...|        1|        |
|       4|            17|             128|        

# Rangos de edades

Vamos a contar los usuarios que usan las bicis por cada rango de edad
Recordamos que, según los datos de BiciMad, los rangos de edades son:

    0 : edad sin determinar
    1 : 0-16 años
    2 : 17-18 años
    3 : 19-26 años
    4 : 27-40 años
    5 : 41-65 años
    6 : 66+ años
    
Suponemos que se distinguen entre menores de 26 y mayores de 27 años porque hasta los 26 años se puede obtener el abono jóven y a partir de los 27 se utiliza el abono normal (más caro) de transporte público

In [55]:
df_a = df.filter((df['ageRange']==0) | (df['ageRange']==1) | (df['ageRange']==2) | (df['ageRange']==3) | (df['ageRange']==4) | (df['ageRange']==5) | (df['ageRange']==6))
df_a.groupBy('ageRange').count().sort('count', ascending = True).show() #esto no modifica el rdd

+--------+-------+
|ageRange|  count|
+--------+-------+
|       6|  14110|
|       2|  37682|
|       1|  43475|
|       3| 212766|
|       5| 661210|
|       4|1093697|
|       0|1194862|
+--------+-------+



Como podemos ver, el rango de edad que más ha utilizador este servicio es el 4, y el que menos el 6. Esto prueba nuestra hipótesis inicial, ya que la gente que puede usar el abono joven en vez del abono general, prefiere usar transporte público a BiciMad

# Residencia

Nos quedamos con los usuarios del tipo 1 (usuarios anuales) y queremos ver de dónde son.
Recordamos que los tipos de usuarios son:

    0: No se ha podido determinar el tipo de usuario
    1: Usuario anual (poseedor de un pase anual)
    2: Usuario ocasional
    3: Trabajador de la empresa

Nos quedamos con los usuarios del tipo 1 y quitamos los usuarios sin codigo postal:

In [56]:
df1 = df.filter(df["user_type"]==1).filter(df.zip_code != '')

Convertimos los códigos postales a enteros.

In [57]:
df1 = df1.withColumn("zip_code",col("zip_code").cast("Integer"))

Agrupamos entre los que son de madrid (zip_code empieza por 28) y los que no. Nos libramos de los valores nulos y hacemos el recuento final:

In [58]:
df1 = df1.withColumn("zip_code",((28000 <= col("zip_code")) & (col("zip_code") < 29000))).withColumnRenamed ("zip_code", "Madrid_residents")


tabla = df1.groupBy('Madrid_residents').count().sort('count', ascending = True)
tabla = tabla.filter((tabla['Madrid_residents']=='true') | (tabla['Madrid_residents']=='false'))
tabla.show()


+----------------+-------+
|Madrid_residents|  count|
+----------------+-------+
|           false|  30892|
|            true|1779617|
+----------------+-------+



# Viajes por usuario

Primero vemos los viajes que hace cada usuario cada día. Lo hacemos creando una lista de listas de la forma [estacion origen, estacion llegada]

Después contamos la cantidad de viajes y finalmente los agrupamos por esa cantidad.

In [62]:
df2 = df.filter(df.user_day_code != '')
df2 = df2.groupBy('user_day_code').agg(collect_list(struct('idunplug_station','idplug_station')).alias('stations'))
df2.show()

+--------------------+--------------------+
|       user_day_code|            stations|
+--------------------+--------------------+
|00001f08f2d92c952...|          [{1, 168}]|
|000030519f5ad9c9b...|          [{53, 56}]|
|0000467a2b4d460a7...|         [{103, 55}]|
|000049d964247d294...|         [{59, 139}]|
|00008d71d44d764ea...|          [{23, 19}]|
|0000987c66b350b3f...|         [{75, 103}]|
|0000a997b548a53c1...|        [{137, 153}]|
|0000ab4ce8b17a0a6...|[{42, 113}, {91, ...|
|0000cd9044767a1f3...|[{157, 93}, {93, ...|
|0000d37943751e6ae...|[{128, 63}, {63, ...|
|0001028318cb5f1cf...|         [{107, 66}]|
|00013b298e099ca64...|          [{64, 64}]|
|00015bc9da21e9e2b...|         [{77, 166}]|
|0001a165f20af3d17...|         [{165, 62}]|
|0001d7326dbe13518...|[{59, 82}, {89, 1...|
|00020f9f14c68c77b...|[{162, 169}, {169...|
|0002173c5a82aa601...|          [{56, 18}]|
|000221dd3607d8ddb...|          [{35, 33}]|
|00023b3e123b9bd9e...|          [{45, 73}]|
|00023c56ecaabf967...|          

Ahora nos interesa saber cuántos viajes realiza en un día cada usuario:

In [63]:
print('numero de viajes por usuario')
trips_len = udf(lambda x: len(x), IntegerType())
dfv = df2.withColumn("stations",trips_len(col("stations"))).withColumnRenamed ("stations", "number_of_trips")
dfv.show()

numero de viajes por usuario


+--------------------+---------------+
|       user_day_code|number_of_trips|
+--------------------+---------------+
|00001f08f2d92c952...|              1|
|000030519f5ad9c9b...|              1|
|0000467a2b4d460a7...|              1|
|000049d964247d294...|              1|
|00008d71d44d764ea...|              1|
|0000987c66b350b3f...|              1|
|0000a997b548a53c1...|              1|
|0000ab4ce8b17a0a6...|              2|
|0000cd9044767a1f3...|              2|
|0000d37943751e6ae...|              2|
|0001028318cb5f1cf...|              1|
|00013b298e099ca64...|              1|
|00015bc9da21e9e2b...|              1|
|0001a165f20af3d17...|              1|
|0001d7326dbe13518...|              3|
|00020f9f14c68c77b...|              2|
|0002173c5a82aa601...|              1|
|000221dd3607d8ddb...|              1|
|00023b3e123b9bd9e...|              1|
|00023c56ecaabf967...|              1|
+--------------------+---------------+
only showing top 20 rows



Vamos a ver cuántos usuarios han realizado 1 viaje por día, 2 viajes, 3 viajes... 

In [64]:
print('recuento de usuarios por numero de viajes')
dfv.groupBy('number_of_trips').count().sort('number_of_trips', ascending = False).show()

recuento de usuarios por numero de viajes


+---------------+-----+
|number_of_trips|count|
+---------------+-----+
|            299|    1|
|            298|    1|
|            270|    1|
|            268|    1|
|            265|    1|
|            260|    1|
|            247|    1|
|            235|    1|
|            232|    1|
|            231|    1|
|            219|    1|
|            211|    1|
|            210|    1|
|            207|    1|
|            202|    2|
|            201|    1|
|            198|    2|
|            196|    1|
|            195|    2|
|            194|    1|
+---------------+-----+
only showing top 20 rows



# Viajes de ida y vuelta

Vamos a comprobar si la gente usa las bicis para viajes de ida y vuelta. Diremos que un usuario ha realizado un viaje de ida y vuelta cuando la estación de salida de uno de sus viajes coincida con la de llegada de otro posterior. Además el usuario debe haber realizado unicamente 2 viajes en un día. Para detectar este tipo de viajes creamos la función 'round_trip(x)':

In [65]:
def round_trip(x):
    result = False
    if len(x) == 2: #solo los viajes con longitud 2 pueden ser de ida y vuelta
        result = x[0][1] == x[1][0] and x[0][0] == x[1][1]
    return result

In [66]:
round_trip_udf = udf(lambda x: round_trip(x), BooleanType())
df3 = df2.withColumn('round_trip', round_trip_udf(df2['stations'])).alias('dfgrt')
df3.show()

+--------------------+--------------------+----------+
|       user_day_code|            stations|round_trip|
+--------------------+--------------------+----------+
|00001f08f2d92c952...|          [{1, 168}]|     false|
|000030519f5ad9c9b...|          [{53, 56}]|     false|
|0000467a2b4d460a7...|         [{103, 55}]|     false|
|000049d964247d294...|         [{59, 139}]|     false|
|00008d71d44d764ea...|          [{23, 19}]|     false|
|0000987c66b350b3f...|         [{75, 103}]|     false|
|0000a997b548a53c1...|        [{137, 153}]|     false|
|0000ab4ce8b17a0a6...|[{42, 113}, {91, ...|     false|
|0000cd9044767a1f3...|[{157, 93}, {93, ...|      true|
|0000d37943751e6ae...|[{128, 63}, {63, ...|      true|
|0001028318cb5f1cf...|         [{107, 66}]|     false|
|00013b298e099ca64...|          [{64, 64}]|     false|
|00015bc9da21e9e2b...|         [{77, 166}]|     false|
|0001a165f20af3d17...|         [{165, 62}]|     false|
|0001d7326dbe13518...|[{59, 82}, {89, 1...|     false|
|00020f9f1

Por último realizamos el recuento:

In [67]:
df3.groupBy('round_trip').count().sort('round_trip').show()

+----------+-------+
|round_trip|  count|
+----------+-------+
|     false|1719318|
|      true| 168597|
+----------+-------+



Como vemos, se realizan 168597 viajes de ida y vuelta en total.

# Duración de los viajes

Vamos a ver la duración de los viajes. Después vamos a categorizarlos para comprobar si son cortos, medios o largos para próximos apartados y vamos a contar cada uno de ellos. La categorización es de la forma:

    Corto: duracion < 500s
    Medio: 500s <= duracion < 1000
    Largo: 1000 <= duracion

Definimos la funcion 'cat_time' para cambiar la etiqueta de los viajes según la duración

In [68]:
def cat_time(x):
    dur = 'Largo'
    if int(x) <500:
        dur = "Corto"
    elif int(x)<1000:
        dur = "Medio"
    return dur       

In [69]:
print('duracion de los viajes')
dfd = df.drop("ageRange","idplug_station","idunplug_station","user_type","zip_code")
cat_time_udf = udf(lambda x: cat_time(x))
dfd2 = dfd.withColumn('travel_time_cat',cat_time_udf(dfd['travel_time']))
dfd2 = dfd2.drop('travel_time')
dfd2.show()

duracion de los viajes


+--------------------+---------------+
|       user_day_code|travel_time_cat|
+--------------------+---------------+
|b00665a845be18ed9...|          Corto|
|65a82ac6353d9e8b2...|          Medio|
|51b7da733980586e1...|          Medio|
|0af09bb0e23045fb7...|          Medio|
|500e6780660a1fb23...|          Corto|
|40816bb3978f9931f...|          Corto|
|ed325bd1941a25394...|          Corto|
|0261bfd1f1bc0a6c5...|          Medio|
|1a172d92c5a570303...|          Corto|
|b7174a2ee06b23318...|          Corto|
|205bd680ee239a407...|          Corto|
|64e4785eca75cef74...|          Medio|
|a0cf408722fbfa76e...|          Medio|
|7f5e1843d0078b858...|          Corto|
|c6beca13cd1d76959...|          Corto|
|4e6a2792bee3876fb...|          Corto|
|ae0f90b2701f9f018...|          Medio|
|7c07e33674df07c02...|          Corto|
|8bc6e5f0d6574cd4e...|          Corto|
|b0ed6e14c736d2935...|          Medio|
+--------------------+---------------+
only showing top 20 rows



In [70]:
print('recuento de viajes por duracion')
dfd2.groupBy('travel_time_cat').count().sort('count',ascending = False).show()

recuento de viajes por duracion


+---------------+-------+
|travel_time_cat|  count|
+---------------+-------+
|          Medio|1364199|
|          Corto|1050310|
|          Largo| 843293|
+---------------+-------+



Los trayectos de duración media son los más comunes, seguidos de los cortos y por último los largos.

# Épocas de mayor y menor uso

Vamos a ver cuáles son los días y los meses de mayor y menor uso de BiciMad. 
La función read_date sirve para quedarnos únicamente con la fecha y read_month para quedarnos con el mes y el año


In [71]:
def read_date(x):
    '''
    La fecha es de la forma aaaa-mm-ddThh..., por lo que nos quedamos con aaaa-mm-dd
    '''
    return x[0][0:10]

In [73]:
dff = union_archivos()
dff = dff.drop('_id','ageRange','user_type','zip_code','idplug_base','idunplug_base','track')
read_date_udf = udf(lambda x: read_date(x))  
#dff = dff.filter(dff.unplug_hourTime != '')
dff2 = dff.withColumn('date',read_date_udf(dff['unplug_hourTime']))
dff2.groupBy('date').count().sort('count',ascending = False).show()
dff2.groupBy('date').count().sort('count',ascending = True).show()

+----------+-----+
|      date|count|
+----------+-----+
|2018-09-27|17103|
|2018-09-26|16174|
|2018-09-25|15164|
|2018-09-19|14942|
|2018-05-10|14642|
|2018-09-13|14563|
|2018-04-25|14529|
|2018-09-11|14392|
|2018-09-20|14328|
|2018-04-26|14242|
|2018-09-12|13998|
|2018-09-14|13967|
|2018-10-03|13884|
|2018-05-09|13807|
|2018-05-11|13793|
|2018-04-18|13756|
|2018-04-19|13725|
|2018-09-18|13709|
|2018-04-24|13683|
|2018-10-04|13604|
+----------+-----+
only showing top 20 rows



+----------+-----+
|      date|count|
+----------+-----+
|2018-05-30|    9|
|2018-05-25|   19|
|2019-01-01|  160|
|2018-08-29|  168|
|2018-01-06| 2037|
|2018-02-04| 2357|
|2018-12-25| 2916|
|2018-01-07| 3122|
|2018-02-05| 3257|
|2018-06-27| 3321|
|2018-04-10| 3670|
|2018-03-01| 3737|
|2018-03-14| 3781|
|2018-03-30| 3926|
|2018-02-28| 4070|
|2018-03-02| 4117|
|2018-12-24| 4229|
|2018-03-10| 4274|
|2018-04-29| 4310|
|2018-03-31| 4441|
+----------+-----+
only showing top 20 rows



Los días que más se usó BiciMad fueron días de septiembre sobre todo, seguidos de abril, mayo y octubre. Tiene sentido, puesto que son épocas en las que las temperaturas son menos extremas y en las que más apetece un paseo en bicicleta.

In [ ]:
dff2.groupBy('date').count().sort('count',ascending = True).show()

Los resultados devuelven que los días que menos se usó BiciMad son el 30 y el 25 de mayo, el 1 de enero de 2019 y el 29 de agosto. Estos resultados creemos que no son del todo correctos y pueden ser días en los que falló el servicio. 

A partir de ahí, podemos observar otros días que parecen mucho más sospechosos de tener poco tráfico. Por ejemplo, el 6 de enero (día de los Reyes Magos) o el 25 de diciembre. Sorprende que el 4 y 5 de febrero tengan tan bajos índices de tráfico, hasta que recordamos que el 4 de febrero nevó en Madrid, sería una locura ir en bicicleta con hielo en la carretera, ¿no?

In [74]:
def get_month(x):
    '''
    La fecha es de la forma aaaa-mm-ddThh..., por lo que nos quedamos con aaaa-mm
    '''
    return x[0][0:7]

Hacemos el mismo proceso para ver qué meses ha habido más trayectos:

In [76]:
get_month_udf = udf(lambda x: get_month(x))  
month_udf = dff.withColumn('month',get_month_udf(dff['unplug_hourTime']))

month_udf.groupBy('month').count().sort('count',ascending = False).show()

+-------+------+
|  month| count|
+-------+------+
|2018-09|368645|
|2018-10|322584|
|2018-07|315045|
|2018-04|297437|
|2018-06|271517|
|2018-01|263702|
|2018-05|262978|
|2018-11|253664|
|2018-02|232103|
|2018-12|229507|
|2018-08|224767|
|2018-03|215693|
|2019-01|   160|
+-------+------+



El mes más concurrido fue septiembre, lo cual es comprensible por el buen tiempo y la vuelta a las clases. El que menos fue marzo, sospechamos que puede tener que ver con que Semana Santa cayó en este mes y por la época de alergias que, aunque la temperatura acompañe, no lo hace tanto a la conducción de las bicis. No tenemos en cuenta enero del 2019 porque sólo contabiliza el día 1 de enero.

# Estaciones más concurridas

Vamos a ver cuáles son las estaciones más y menos concurridas. Primero como estación de llegada del viaje:

In [77]:
dff.groupBy('idplug_station').count().sort('count',ascending = False).show()
dff.groupBy('idplug_station').count().sort('count',ascending = True).show()

+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           135|50537|
|            43|48124|
|           175|48014|
|           163|46716|
|           129|45538|
|            57|44300|
|           149|42010|
|           168|39838|
|           160|35679|
|           162|35244|
|            58|34071|
|            83|33894|
|            49|33739|
|             9|33522|
|            90|33212|
|            64|31579|
|           132|31115|
|             1|31036|
|            19|30758|
|           128|30576|
+--------------+-----+
only showing top 20 rows



+--------------+-----+
|idplug_station|count|
+--------------+-----+
|          2008|   17|
|            37| 3949|
|            28| 4346|
|            24| 5049|
|           119| 5267|
|            23| 5508|
|            88| 5848|
|            29| 6106|
|           144| 6514|
|           173| 6864|
|           120| 7413|
|           104| 8130|
|           150| 8147|
|            61| 8361|
|           105| 8623|
|           101| 8680|
|           138| 8910|
|           107| 9193|
|           111| 9557|
|           151| 9629|
+--------------+-----+
only showing top 20 rows



La estación a la que llegan más bicicletas es la 135, y corresponde a una estación entre Nuevos Ministerios y Gregorio Marañón, y de la que menos la 2008.

A continuación vemos la más concurrida como estación de origen:

In [78]:
dff.groupBy('idunplug_station').count().sort('count',ascending = False).show()
dff.groupBy('idunplug_station').count().sort('count',ascending = True).show()

+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             135|49188|
|             175|46718|
|              43|46634|
|             163|46604|
|             129|45892|
|              57|45080|
|             149|41878|
|             168|39041|
|             160|35255|
|              58|34797|
|             162|34593|
|              83|34109|
|               9|33941|
|              90|33542|
|              49|33173|
|              64|32526|
|              19|31405|
|               1|31207|
|             132|30969|
|             128|30698|
+----------------+-----+
only showing top 20 rows



+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|            2008|   17|
|              37| 3968|
|              28| 4502|
|              24| 5098|
|              88| 5289|
|             119| 5405|
|              23| 5520|
|              29| 6203|
|             173| 6883|
|             144| 6955|
|             120| 7170|
|             150| 8034|
|              61| 8086|
|             104| 8416|
|             105| 8688|
|             138| 8970|
|             146| 9598|
|             151| 9693|
|             147| 9770|
|             101| 9778|
+----------------+-----+
only showing top 20 rows



La estación de la que salen más bicicletas es la 135 y de la que menos la 2008. 

Es muy interesante que en ambos casos las estaciones son las mismas. 

# Trayectos según rangos de edad

Vamos a comparar la longitud de los trayectos en los diferentes rangos de edad. 

In [79]:
#Vemos que rango de edad hace los trayectos de mayor duracion
df8 = df.drop("idplug_station","idunplug_station","user_type","zip_code")

#Categorizamos la duracion de los trayectos y dejamos solo los viajes largos:
cat_time_udf = udf(lambda x: cat_time(x))
df8 = df8.withColumn('travel_time',cat_time_udf(df8['travel_time']))

In [80]:
#quitamos el rango de edad 0 ya que se corresponde con las edades desconocidas
df_largo = df8.filter((df8["travel_time"]=='Largo')&(df8["ageRange"]!='0'))
df_largo2 = df_largo.groupBy('ageRange').count().sort('count',ascending = False).withColumnRenamed('count','count_largo')
df_largo2.show()

+--------+-----------+
|ageRange|count_largo|
+--------+-----------+
|       4|     249720|
|       5|     175442|
|       3|      46778|
|       1|      20170|
|       2|       8183|
|       6|       3793|
+--------+-----------+



El rango de edad que realiza menos viajes largos son los mayores de 66, lo cual tiene sentido, y el que más los de 27 a 40 años.

In [81]:
df_medio = df8.filter((df8["travel_time"]=='Medio')&(df8["ageRange"]!='0'))
df_medio2 = df_medio.groupBy('ageRange').count().sort('count',ascending = False).withColumnRenamed('count','count_medio')
df_medio2.show()

+--------+-----------+
|ageRange|count_medio|
+--------+-----------+
|       4|     482713|
|       5|     280465|
|       3|      87940|
|       2|      15048|
|       6|       5871|
|       1|       3822|
+--------+-----------+



El rango de edad que realiza menos viajes medios son los de 0 a 16 y el que más los de 27-40 años.

In [82]:
df_corto = df8.filter((df8["travel_time"]=='Corto')&(df8["ageRange"]!='0'))
df_corto2 = df_corto.groupBy('ageRange').count().sort('count',ascending = False).withColumnRenamed('count','count_corto')
df_corto2.show()

+--------+-----------+
|ageRange|count_corto|
+--------+-----------+
|       4|     361264|
|       5|     205303|
|       3|      78048|
|       1|      19483|
|       2|      14451|
|       6|       4446|
+--------+-----------+



El rango de edad que realiza menos viajes largos son los mayores de 66 y el que más los de 27 a 40 años (igual que en los viajes largos).

Construimos un dataFrame que nos junte la información para poder compararla.

In [83]:
df_aux1 = df_medio2.join(df_largo2, on=['ageRange'], how='left_outer')
df_aux2 = df_corto2.join(df_aux1, on=['ageRange'],how='left_outer')

df_aux2.sort('ageRange').show()

+--------+-----------+-----------+-----------+
|ageRange|count_corto|count_medio|count_largo|
+--------+-----------+-----------+-----------+
|       1|      19483|       3822|      20170|
|       2|      14451|      15048|       8183|
|       3|      78048|      87940|      46778|
|       4|     361264|     482713|     249720|
|       5|     205303|     280465|     175442|
|       6|       4446|       5871|       3793|
+--------+-----------+-----------+-----------+



Es cuioso que en el rango 1 (0-16 años) abunden tanto los viajes largos en comparación a los medios. En general, los viajes medios son algo más abundantes que los cortos y los largos son de los que menos cantidad hay.

# Estudio del rango de edad

Vamos a realizar un estudio de los usuarios para cada rango de edad, para obtener la estación más concurrida para cada uno y de donde son residentes dichos usuarios principalmente .

In [84]:
def origen(datos,i):
    dfe_aux = datos.filter(datos["ageRange"]==i)

    #Vamos a ver de qué estación salen los usuarios pertenecientes al grupo de edad i: 
    dfe_aux.groupBy('idunplug_station').count().sort('count',ascending = False).show() 
    estacion,count = dfe_aux.groupBy('idunplug_station').count().sort('count',ascending = False).head()  #La estación más usada

    dfe_aux = datos.filter(datos["idunplug_station"]==estacion).filter(datos["zip_code"]!='')
    dfe_aux.groupBy('zip_code').count().sort('count',ascending = False).show()
    codigo, count = dfe_aux.groupBy('zip_code').count().sort('count',ascending = False).head()
    return (estacion,codigo)

In [85]:
def destino(datos, i):
    dfe_aux = datos.filter(datos["ageRange"]==i)

    #Vamos a ver a qué estación llegan los usuarios pertenecientes al grupo de edad i: 
    dfe_aux.groupBy('idplug_station').count().sort('count',ascending = False)
    estacion,count = dfe_aux.groupBy('idplug_station').count().sort('count',ascending = False).head()  #La estación más usada

    dfe_aux = datos.filter(datos["idplug_station"]==estacion).filter(datos["zip_code"]!='')
    dfe_aux.groupBy('zip_code').count().sort('count',ascending = False)
    codigo, count = dfe_aux.groupBy('zip_code').count().sort('count',ascending = False).head()
    return (estacion,codigo)

In [86]:
print("Este apartado tarda")
for i in [1,2,3,4,5,6]:
    print(f"Vamos a estudiar el rango de edad {i}:")
    
    est,cod = origen(df,i)
    print(f"La estación de origen que toma más comunmente es: {est} y los usuarios provienen principalmente del código postal {cod}")
        
    est,cod = destino(df,i)
    print(f"La estación de destino que usada más frecuentemente es: {est} y los usuarios provienen principalmente del código postal {cod}")
    

Vamos a estudiar el rango de edad 1:


+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             132| 1191|
|             135| 1089|
|             129| 1028|
|             162|  833|
|             108|  770|
|             133|  766|
|              64|  762|
|             168|  742|
|              43|  701|
|              69|  692|
|              82|  649|
|              79|  640|
|              91|  615|
|             175|  577|
|              49|  554|
|             160|  546|
|              78|  544|
|              73|  542|
|             128|  510|
|             136|  502|
+----------------+-----+
only showing top 20 rows



+--------+-----+
|zip_code|count|
+--------+-----+
|   28008| 4324|
|   28005| 1338|
|   28012|  903|
|   28003|  858|
|   28045|  672|
|   28015|  629|
|   28004|  523|
|   28013|  312|
|   28011|  304|
|   28010|  271|
|   28014|  234|
|   28007|  208|
|   28002|  185|
|   28039|  178|
|   28001|  169|
|   28009|  158|
|   28019|  145|
|   28922|  125|
|   06800|  122|
|   28026|  115|
+--------+-----+
only showing top 20 rows



La estación de origen que toma más comunmente es: 132 y los usuarios provienen principalmente del código postal 28008


+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           129| 1036|
|            79|  704|
|           168|  671|
|            78|  663|
|            75|  659|
|           109|  653|
|           108|  640|
|           175|  639|
|           160|  563|
|            42|  559|
|           115|  556|
|            34|  551|
|           133|  521|
|            49|  520|
|            77|  520|
|           145|  518|
|            83|  500|
|            43|  497|
|           136|  471|
|           162|  464|
+--------------+-----+
only showing top 20 rows



...

+--------+-----+
|zip_code|count|
+--------+-----+
|   28001| 2297|
|   28007| 1343|
|   28012| 1026|
|   28004|  867|
|   28009|  720|
|   28005|  608|
|   28010|  414|
|   28045|  400|
|   28006|  358|
|   28028|  327|
|   28014|  315|
|   28016|  315|
|   28020|  290|
|   28015|  284|
|   28013|  221|
|   28002|  219|
|   28003|  193|
|   28008|  121|
|   28023|   82|
|   28000|   66|
+--------+-----+
only showing top 20 rows



La estación de origen que toma más comunmente es: 113 y los usuarios provienen principalmente del código postal 28001


+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           160|  379|
|            30|  330|
|           164|  295|
|           163|  291|
|           135|  278|
|           113|  263|
|            83|  256|
|           104|  248|
|           103|  235|
|           161|  230|
|            97|  208|
|            44|  206|
|           168|  201|
|            64|  201|
|            27|  197|
|            65|  182|
|            13|  181|
|            74|  180|
|            38|  179|
|            73|  176|
+--------------+-----+
only showing top 20 rows



+--------+-----+
|zip_code|count|
+--------+-----+
|   28015| 5139|
|   28003| 4083|
|   28004| 1688|
|   28010| 1286|
|   28008| 1286|
|   28005| 1120|
|   28012|  799|
|   28007|  629|
|   28020|  576|
|   28040|  416|
|   28045|  379|
|   28009|  313|
|   28013|  288|
|   28006|  272|
|   28028|  256|
|   28039|  253|
|   28002|  220|
|   28046|  201|
|   28043|  200|
|   28037|  176|
+--------+-----+
only showing top 20 rows



La estación de destino que usada más frecuentemente es: 160 y los usuarios provienen principalmente del código postal 28015
